# Lab 3: AgentCore Gateway로 도구를 에이전트에 안전하게 연결

## 개요

이 랩에서는 Amazon Bedrock Gateway를 사용하여 조직에서 사용 가능한 도구를 이커머스 고객 지원 에이전트와 통합하는 방법을 배웁니다.

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro)는 애플리케이션이 대형 언어 모델(LLM)에 도구와 컨텍스트를 제공하는 방법을 표준화하는 개방형 프로토콜입니다.

[Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)를 사용하면 개발자는 몇 줄의 코드만으로 API, Lambda 함수 및 기존 서비스를 MCP 호환 도구로 변환하고 Gateway 엔드포인트를 통해 에이전트에서 사용할 수 있습니다.

**워크숍 여정:**

- **Lab 1 (완료):** 에이전트 프로토타입 생성 - 기능적인 이커머스 고객 지원 에이전트 구축
- **Lab 2 (완료):** 메모리로 강화 - 대화 컨텍스트 및 개인화 추가
- **Lab 3 (현재):** Gateway & Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4:** 프로덕션 배포 - AgentCore Runtime으로 관측성 확보
- **Lab 5:** 사용자 인터페이스 구축 - 고객 대상 애플리케이션 생성

### AgentCore Gateway & 도구 공유가 중요한 이유

현재 상태 (Lab 1-2): 각 에이전트가 자체 도구 사본을 가지고 있습니다. 실제로는 확장 가능하지 않고 다음과 같은 문제를 야기합니다:

- 다른 에이전트 간 코드 중복
- 일관성 없는 도구 동작 및 유지 관리 오버헤드
- 중앙화된 보안 또는 접근 제어 없음
- 여러 사용 사례로 확장하기 어려움

이 랩 후에는 다음을 제공할 수 있는 중앙화된 재사용 가능한 도구를 갖게 됩니다:

- 이커머스 고객 지원 에이전트 (현재 사용 사례)
- 판매 에이전트 (동일한 상품 정보 및 고객 데이터 필요)
- 재고 에이전트 (동일한 상품 정보 및 보증 확인 필요)
- 반품 처리 에이전트 (반품 정책 및 고객 프로필 필요)

그리고 기타 사용 사례들.

### AgentCore Identity로 보안 인증 추가

또한 AgentCore Gateway는 인바운드 및 아웃바운드 연결을 안전하게 인증해야 합니다. [AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html)는 Okta, Entra, Amazon Cognito와 같은 표준 ID 공급자를 지원하면서 AWS 서비스 및 Slack, Zoom과 같은 타사 애플리케이션 전반에 걸쳐 원활한 에이전트 ID 및 액세스 관리를 제공합니다. 이 랩에서는 AgentCore Gateway가 AgentCore Identity와 통합하여 인바운드 및 아웃바운드 인증을 통해 보안 연결을 제공하는 방법을 살펴보겠습니다.

인바운드 인증의 경우 AgentCore Gateway는 호출 중에 전달된 OAuth 토큰을 분석하여 게이트웨이의 도구에 대한 액세스를 허용하거나 거부할지 결정합니다. 도구가 외부 리소스에 액세스해야 하는 경우 AgentCore Gateway는 API 키, IAM 또는 OAuth 토큰을 통해 아웃바운드 인증을 사용하여 외부 리소스에 대한 액세스를 허용하거나 거부할 수 있습니다.

## Lab 3을 위한 아키텍처

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*반품 자격 검증 도구가 이제 보안 신원 기반 액세스 제어와 함께 AgentCore Gateway에서 중앙화되었습니다. 여러 에이전트와 사용 사례가 동일한 도구를 안전하게 공유할 수 있습니다. 기존 도구들(`process_return()`, `process_exchange()`, `web_search()`)은 이커머스 고객 지원 사용 사례에 특화되어 있으므로 로컬 도구로 유지됩니다.*

### 주요 기능
- **AWS Lambda 함수 원활하게 통합:** 이 예제는 기존 AWS Lambda 함수와 에이전트를 통합하여 Amazon Bedrock AgentCore Gateway를 사용하여 항목의 반품 자격을 확인하는 방법을 보여줍니다.
- **인바운드 인증으로 Gateway 엔드포인트 보안:** 유효한 JWT 토큰을 제공하는 에이전트만 엔드포인트에 연결하여 도구를 사용할 수 있습니다
- **MCP 엔드포인트를 사용하도록 에이전트 구성:** 에이전트가 유효한 JWT 토큰을 가져와서 AgentCore Gateway에서 제공하는 MCP 엔드포인트에 연결하는 데 사용합니다



### 📋 배포 상태 확인 

AWS 인프라가 제대로 배포되었는지 확인하려면 아래 셀을 실행하세요:

In [ ]:
# 배포된 AWS 리소스 확인
import boto3
import json
from datetime import datetime

print("🔍 배포된 AWS 리소스 확인 중...")
print("=" * 50)

try:
    # CloudFormation 스택 상태 확인
    cf_client = boto3.client('cloudformation')
    
    stacks = [
        'EcommerceCustomerSupportStackInfra',
        'EcommerceCustomerSupportStackCognito'
    ]
    
    print("📚 CloudFormation 스택 상태:")
    for stack_name in stacks:
        try:
            response = cf_client.describe_stacks(StackName=stack_name)
            stack = response['Stacks'][0]
            status = stack['StackStatus']
            created = stack['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
            
            status_emoji = "✅" if "COMPLETE" in status else "⏳" if "PROGRESS" in status else "❌"
            print(f"   {status_emoji} {stack_name}: {status} (생성: {created})")
        except:
            print(f"   ❌ {stack_name}: 스택을 찾을 수 없습니다")
    
    print()
    
    # SSM 파라미터 확인
    ssm_client = boto3.client('ssm')
    try:
        response = ssm_client.get_parameters_by_path(
            Path='/app/ecommerce/agentcore',
            Recursive=True
        )
        
        print(f"📋 SSM 파라미터 ({len(response['Parameters'])}개):")
        for param in sorted(response['Parameters'], key=lambda x: x['Name']):
            name = param['Name'].split('/')[-1]
            print(f"   ✅ {name}")
        print()
        
    except Exception as e:
        print(f"   ❌ SSM 파라미터 조회 실패: {str(e)}")
    
    # Lambda 함수 확인
    lambda_client = boto3.client('lambda')
    try:
        functions = lambda_client.list_functions()['Functions']
        ecommerce_functions = [f for f in functions if 'EcommerceCustomer' in f['FunctionName']]
        
        print(f"🔧 Lambda 함수 ({len(ecommerce_functions)}개):")
        for func in ecommerce_functions:
            print(f"   ✅ {func['FunctionName']} ({func['Runtime']})")
        print()
        
    except Exception as e:
        print(f"   ❌ Lambda 함수 조회 실패: {str(e)}")
    
    print("🎯 준비 완료! 이제 Step 1부터 진행하세요.")
    
except Exception as e:
    print(f"❌ 리소스 확인 중 오류: {str(e)}")
    print("💡 AWS 자격 증명을 확인하고 인프라 배포를 다시 시도해주세요.")

print("=" * 50)

## Step 1: 필요한 라이브러리 설치 및 가져오기

In [ ]:
# 라이브러리 가져오기
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import os
import sys
import boto3
import json
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

# 프로젝트 루트 경로를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import sys; sys.path.insert(0, '..'); from src.helpers.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret

sts_client = boto3.client('sts')

# AWS 계정 세부 정보 가져오기
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ 라이브러리 가져오기 성공!")
print(f"📂 프로젝트 루트: {project_root}")

## Step 2: 에이전트에 기존 이커머스 데이터에 액세스할 수 있는 도구 제공

AgentCore Gateway는 세 가지 주요 방법으로 에이전트 도구 통합을 단순화합니다:

| 통합 방식 | 설명 | 사용 사례 |
|-----------|------|-----------|
| **MCP 지원** | MCP 표준을 통해 노출하여 모든 에이전트 프레임워크와 호환 | 크로스 플랫폼 도구 공유 |
| **REST 통합** | 기존 REST 서비스를 Gateway 대상으로 추가하여 에이전트 도구로 변환 | 레거시 API 통합 |
| **Lambda 유연성** | Lambda 함수를 MCP 엔드포인트로 노출 | 커스텀 비즈니스 로직 |

### 데이터 흐름

```
┌─────────────────┐      ┌────────────────────┐      ┌─────────────────────┐
│  Strands Agent  │─────▶│  AgentCore Gateway │─────▶│   Lambda Function   │
│                 │      │                    │      │                     │
│  MCP Client     │◀─────│  JWT 인증 검증     │◀─────│  반품 자격 검증     │
└─────────────────┘      └────────────────────┘      └─────────────────────┘
        │                         │                           │
        │                         ▼                           │
        │              ┌────────────────────┐                 │
        │              │  event 객체 전달:  │                 │
        │              │  • tool_name       │                 │
        │              │  • parameters      │                 │
        │              └────────────────────┘                 │
        │                                                     │
        └─────────────────── 응답 반환 ◀──────────────────────┘
```

### Lambda 함수 구조

AgentCore Gateway는 Lambda 함수를 호출할 때 `event` 객체에 다음 정보를 전달합니다:

```python
event = {
    "tool_name": "check_return_eligibility",  # 호출된 도구 이름
    "parameters": {                            # 에이전트가 전달한 매개변수
        "order_number": "KS-2024-001234",
        "customer_id": "customer_ecommerce_001"
    }
}
```

Lambda 함수에서는 이 정보를 추출하여 처리합니다:

[Lambda 함수 전체 코드](../setup/lambda/return_eligibility_check.py)

```python
def get_tool_name(event: Dict[str, Any]) -> str:
    """이벤트에서 도구 이름을 추출합니다."""
    return event.get('tool_name', '')

def get_named_parameter(event: Dict[str, Any], name: str) -> str:
    """이벤트에서 지정된 매개변수를 추출합니다."""
    parameters = event.get('parameters', {})
    return parameters.get(name, '')

def lambda_handler(event, context):
    tool_name = get_tool_name(event)
    
    if tool_name == "check_return_eligibility":
        order_number = get_named_parameter(event, "order_number")
        customer_id = get_named_parameter(event, "customer_id")
        
        result = check_return_eligibility(order_number, customer_id)
        return {"statusCode": 200, "body": json.dumps(result)}
```

### 주요 포인트

1. **도구 이름 라우팅**: 하나의 Lambda에서 여러 도구를 처리할 수 있음 (`tool_name`으로 분기)
2. **매개변수 전달**: Gateway가 에이전트의 도구 호출 인자를 `parameters` 딕셔너리로 전달
3. **응답 형식**: `statusCode`와 `body`를 포함한 표준 Lambda 응답 형식 사용

## Step 3: 도구를 MCP로 변환

AgentCore Gateway를 사용하면 여러 에이전트에서 공유할 도구를 MCP 서버로 중앙화할 수 있습니다.

이 랩에서는 반품 자격 검증 로직을 Lambda 함수로 구현하여 AgentCore Gateway를 통해 노출합니다:

[반품 자격 검증 Lambda](../setup/lambda/return_eligibility_check.py)

```python
def check_return_eligibility(order_number: str, customer_id: str) -> Dict[str, Any]:
    """
    패션/뷰티 상품의 반품 자격을 자동으로 검증합니다.
    
    검증 항목:
    - 주문 존재 여부
    - 고객 ID 일치 여부  
    - 반품 기간 (배송일로부터 7일, VIP는 +2일)
    - 상품 상태 (택 제거, 착용 흔적, 미개봉 등)
    
    Returns:
        eligible: 반품 가능 여부
        reason: 불가 사유 (해당 시)
        estimated_refund: 예상 환불액
        conditions: 반품 조건 목록
    """
    # 실제 환경에서는 DynamoDB나 RDS에서 조회
    # VIP 등급, 배송일, 상품 상태 등을 종합 판단
    return eligibility_result
```

### 로컬 도구 vs Gateway 도구

| 구분 | 로컬 도구 | Gateway 도구 |
|------|-----------|--------------|
| 위치 | 에이전트 코드 내 | 중앙 Lambda |
| 공유 | 불가 (복사 필요) | 여러 에이전트에서 공유 |
| 업데이트 | 각 에이전트 수정 | 한 곳만 수정 |
| 인증 | 없음 | JWT 기반 인증 |
| 예시 | `process_return`, `web_search` | `check_return_eligibility` |

## Step 4: 함수 정의 메타데이터 생성

AgentCore Gateway가 Lambda 함수를 MCP 도구로 노출하려면 **도구 스키마**가 필요합니다. 이 스키마는 에이전트에게 도구의 용도와 필요한 매개변수를 알려줍니다.

도구 스키마 파일: [setup/lambda/api_spec.json](../setup/lambda/api_spec.json)

```json
[{
    "name": "check_return_eligibility",
    "description": "패션/뷰티 상품의 반품 자격을 자동으로 검증합니다. 주문일, 배송일, 상품 상태, VIP 등급 등을 종합적으로 판단하여 반품 가능 여부를 결정합니다.",
    "inputSchema": {
        "type": "object",
        "properties": {
            "order_number": {
                "type": "string",
                "description": "주문번호 (예: 'KS-2024-001234')"
            },
            "customer_id": {
                "type": "string", 
                "description": "고객 ID (예: 'customer_ecommerce_001')"
            }
        },
        "required": ["order_number", "customer_id"]
    }
}]
```

### 스키마 구조 설명

| 필드 | 설명 |
|------|------|
| `name` | 도구 이름 (Lambda의 `tool_name`과 매칭) |
| `description` | LLM이 도구 선택 시 참고하는 설명 |
| `inputSchema` | JSON Schema 형식의 매개변수 정의 |
| `properties` | 각 매개변수의 타입과 설명 |
| `required` | 필수 매개변수 목록 |

> **💡 팁**: `description`을 상세하게 작성하면 LLM이 적절한 상황에서 도구를 더 잘 선택합니다.

## Step 5. AgentCore Gateway 생성

이제 Lambda 함수를 MCP 호환 엔드포인트로 노출하는 AgentCore Gateway를 생성해보겠습니다.

호출자가 도구를 호출할 권한이 있는지 확인하기 위해 인바운드 인증을 구성해야 합니다.

인바운드 인증은 MCP 서버의 표준인 OAuth 인증을 사용하여 작동합니다. OAuth를 사용하면 클라이언트 애플리케이션이 Gateway를 사용하기 전에 OAuth 인증자로 인증해야 합니다. 클라이언트는 런타임에 사용되는 액세스 토큰을 받습니다.

OAuth 검색 서버와 클라이언트 ID를 지정해야 합니다. 워크숍과 함께 제공된 Cloudformation은 이미 Cognito UserPool과 UserPoolClient를 프로비저닝했으며 전용 SSM 매개변수에 검색 URL과 클라이언트 ID를 저장했습니다.

In [ ]:
gateway_name = "ecommerce-gw"

print("machine_client_id", get_ssm_parameter("/app/ecommerce/agentcore/machine_client_id"))
print("cognito_discovery_url", get_ssm_parameter("/app/ecommerce/agentcore/cognito_discovery_url"))

# 이커머스 전용 인증 설정
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            get_ssm_parameter("/app/ecommerce/agentcore/machine_client_id")
        ],
        "discoveryUrl": get_ssm_parameter("/app/ecommerce/agentcore/cognito_discovery_url")
    }
}

try:
    # 새 게이트웨이 생성
    print(f"리전 {REGION}에서 이름이 {gateway_name}인 게이트웨이 생성 중")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=get_ssm_parameter("/app/ecommerce/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="이커머스 고객 지원 AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/ecommerce/agentcore/gateway_id", gateway_id)

    print(f"✅ 게이트웨이가 ID {gateway_id}로 성공적으로 생성되었습니다")

except Exception as e:
    # 게이트웨이가 존재하는 경우 SSM에서 기존 게이트웨이 ID 수집
    try:
        existing_gateway_id = get_ssm_parameter("/app/ecommerce/agentcore/gateway_id")
        print(f"ID {existing_gateway_id}인 기존 게이트웨이를 찾았습니다")
        
        # 기존 게이트웨이 세부 정보 가져오기
        gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
        gateway = {
            "id": existing_gateway_id,
            "name": gateway_response["name"],
            "gateway_url": gateway_response["gatewayUrl"],
            "gateway_arn": gateway_response["gatewayArn"],
        }
        gateway_id = gateway['id']
    except Exception as inner_e:
        print(f"❌ 게이트웨이 생성/조회 실패: {str(e)}")
        print(f"   추가 오류: {str(inner_e)}")
        raise

## Step 6. Lambda 함수 대상 추가

이제 Step 4에서 정의한 도구 스키마([setup/lambda/api_spec.json](../setup/lambda/api_spec.json))를 사용하여 AgentCore Gateway에 Lambda 대상을 생성합니다.

### Gateway 대상(Target) 구조

```
┌─────────────────────────────────────────────────────────────┐
│                    AgentCore Gateway                         │
│                                                             │
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────┐ │
│  │  Lambda Target  │  │   REST Target   │  │  MCP Target │ │
│  │  (반품 자격)    │  │   (결제 API)    │  │  (외부 서비스)│ │
│  └────────┬────────┘  └────────┬────────┘  └──────┬──────┘ │
│           │                    │                   │        │
│           └────────────────────┼───────────────────┘        │
│                                │                            │
│                    단일 MCP 엔드포인트 URL                   │
└────────────────────────────────┼────────────────────────────┘
                                 │
                                 ▼
                         에이전트 (MCP Client)
```

### 대상 구성 요소

| 구성 요소 | 설명 |
|-----------|------|
| `name` | 대상 식별 이름 (예: `EcommerceLambdaTarget`) |
| `targetConfiguration` | Lambda ARN 및 도구 스키마 |
| `credentialProviderConfigurations` | 인증 방식 (IAM Role, API Key, OAuth) |

### 자격 증명 공급자 유형

```python
# Gateway가 Lambda를 호출할 때 사용하는 인증 방식
credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
```

| 유형 | 설명 | 사용 사례 |
|------|------|-----------|
| `GATEWAY_IAM_ROLE` | Gateway의 IAM Role 사용 | Lambda, AWS 서비스 |
| `API_KEY` | API 키 인증 | 외부 REST API |
| `OAUTH` | OAuth 토큰 인증 | 타사 서비스 (Slack, GitHub 등) |

> **💡 핵심 포인트**: 여러 대상을 하나의 Gateway에 연결하면, 에이전트는 단일 MCP URL만으로 모든 도구에 접근할 수 있습니다.

In [ ]:
def load_api_spec(file_path: str) -> list:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    if not isinstance(data, list):
        raise ValueError("JSON 파일에 리스트가 예상됩니다")
    return data

# API 스펙 파일 경로 후보들
api_spec_candidates = [
    "../setup/lambda/api_spec.json",
    os.path.join(project_root, "setup/lambda/api_spec.json"),
]

api_spec_file = None
for candidate in api_spec_candidates:
    if os.path.exists(candidate):
        api_spec_file = candidate
        break

if api_spec_file is None:
    print(f"❌ API 명세 파일을 다음 경로에서 찾을 수 없습니다:")
    for candidate in api_spec_candidates:
        print(f"   - {candidate}")
else:
    api_spec = load_api_spec(api_spec_file)
    print(f"✅ API 명세 파일 로드 완료: {api_spec_file}")
    print(f"\n📋 도구 스키마 내용:")
    print(json.dumps(api_spec, indent=2, ensure_ascii=False))

    # SSM 파라미터 값 출력
    lambda_arn = get_ssm_parameter("/app/ecommerce/agentcore/lambda_arn")
    print(f"\n🔧 SSM 파라미터:")
    print(f"   Lambda ARN: {lambda_arn}")
    print(f"   Gateway ID: {gateway_id}")

    target_name = "EcommerceLambdaTarget"
    
    # 대상 생성 시도
    try:
        lambda_target_config = {
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        }

        credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

        create_target_response = gateway_client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name=target_name,
            description="이커머스 반품 자격 검증 Lambda 대상",
            targetConfiguration=lambda_target_config,
            credentialProviderConfigurations=credential_config,
        )

        print(f"\n✅ 게이트웨이 대상 생성됨: {create_target_response['targetId']}")

    except gateway_client.exceptions.ConflictException:
        print(f"\n✅ 기존 대상 '{target_name}'이 이미 존재합니다.")
        try:
            targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gateway_id)
            for target in targets_response.get('items', []):
                if target.get('name') == target_name:
                    print(f"   Target ID: {target.get('targetId')}")
                    print(f"   Status: {target.get('status', 'N/A')}")
                    break
        except:
            pass
            
    except Exception as e:
        print(f"\n❌ 게이트웨이 대상 생성 오류: {str(e)}")

## Step 7: 새로운 MCP 기반 도구를 지원 에이전트에 추가

이제 Cognito 인증 토큰을 Strands SDK의 MCPClient에 통합하여 보안 MCP 연결을 설정합니다.

### OAuth 2.0 Client Credentials Flow

```
┌──────────────┐                    ┌─────────────────┐                    ┌──────────────────┐
│   Agent      │                    │  Cognito        │                    │  AgentCore       │
│  (MCP Client)│                    │  (OAuth Server) │                    │  Gateway         │
└──────┬───────┘                    └────────┬────────┘                    └────────┬─────────┘
       │                                     │                                      │
       │  1. Token Request                   │                                      │
       │  (client_id, client_secret, scope)  │                                      │
       │────────────────────────────────────▶│                                      │
       │                                     │                                      │
       │  2. Access Token (JWT)              │                                      │
       │◀────────────────────────────────────│                                      │
       │                                     │                                      │
       │  3. MCP Request + Bearer Token      │                                      │
       │─────────────────────────────────────┼─────────────────────────────────────▶│
       │                                     │                                      │
       │                                     │  4. JWKS 공개키 조회                  │
       │                                     │◀─────────────────────────────────────│
       │                                     │                                      │
       │                                     │  5. 공개키 반환                       │
       │                                     │─────────────────────────────────────▶│
       │                                     │                                      │
       │                                     │  6. JWT 검증 (Gateway에서 수행)       │
       │                                     │  • 서명 유효성                        │
       │                                     │  • 만료 시간 (exp)                    │
       │                                     │  • client_id 확인                    │
       │                                     │  • scope 확인                        │
       │                                     │                                      │
       │  7. Tool Response                   │                                      │
       │◀────────────────────────────────────┼──────────────────────────────────────│
       │                                     │                                      │
```

### JWT 검증 역할 구분

| 구성 요소 | 역할 |
|-----------|------|
| **Cognito Resource Server** | Scope 정의만 담당 (검증 안 함) |
| **Cognito Token Endpoint** | JWT 토큰 발급 |
| **AgentCore Gateway** | JWT 토큰 검증 수행 |

### Gateway의 JWT 검증 항목

```python
# Gateway 생성 시 설정한 인증 구성
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [client_id],         # 허용된 클라이언트 ID
        "discoveryUrl": cognito_discovery_url  # JWKS 공개키 위치
    }
}
```

Gateway는 다음 항목을 검증합니다:
1. **서명 검증**: `discoveryUrl`의 JWKS 공개키로 JWT 서명 확인
2. **만료 확인**: `exp` 클레임으로 토큰 유효 기간 확인
3. **클라이언트 확인**: `allowedClients` 목록과 토큰의 `client_id` 비교
4. **Scope 확인**: 토큰에 필요한 권한이 포함되어 있는지 확인

### 인증 구성 요소

| 구성 요소 | 설명 | SSM 파라미터 |
|-----------|------|--------------|
| `client_id` | Cognito 앱 클라이언트 ID | `/app/ecommerce/agentcore/machine_client_id` |
| `client_secret` | 앱 클라이언트 시크릿 | Cognito에서 직접 조회 |
| `scope` | 요청 권한 범위 | `/app/ecommerce/agentcore/cognito_auth_scope` |
| `token_url` | 토큰 엔드포인트 URL | `/app/ecommerce/agentcore/cognito_token_url` |

### 토큰 요청 함수

```python
def get_token(client_id, client_secret, scope, url):
    """OAuth 2.0 Client Credentials Grant로 액세스 토큰을 요청합니다."""
    data = {
        "grant_type": "client_credentials",  # 서버 간 인증에 사용
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": scope,                       # 요청할 권한 범위
    }
    response = requests.post(url, data=data)
    return response.json()  # {"access_token": "...", "expires_in": 3600, ...}
```

> **💡 핵심 포인트**: Cognito Resource Server는 "어떤 scope가 존재하는지" 정의하고, **AgentCore Gateway**가 실제로 JWT 토큰을 검증합니다. Client Credentials Grant는 사용자 개입 없이 서버 간 인증에 사용되는 OAuth 플로우입니다.

In [ ]:
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,
        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

## Step 7.1. 보안 MCP 클라이언트 객체 설정

이 단계에서는 AgentCore Gateway에 안전하게 연결하기 위한 MCP 클라이언트를 설정합니다.

### 설정 과정

```
┌─────────────────────────────────────────────────────────────────────────┐
│  1. SSM에서 OAuth 자격 증명 조회                                         │
│     • client_id, client_secret, scope, token_url                        │
└────────────────────────────┬────────────────────────────────────────────┘
                             ▼
┌─────────────────────────────────────────────────────────────────────────┐
│  2. Cognito 토큰 엔드포인트에 액세스 토큰 요청                            │
│     POST {token_url} → { access_token, expires_in }                     │
└────────────────────────────┬────────────────────────────────────────────┘
                             ▼
┌─────────────────────────────────────────────────────────────────────────┐
│  3. MCPClient 생성 (Bearer 토큰 포함)                                    │
│     Authorization: Bearer {access_token}                                │
└─────────────────────────────────────────────────────────────────────────┘
```

### SSM 파라미터 설명

| 파라미터 | 설명 | 예시 |
|----------|------|------|
| `machine_client_id` | Cognito 앱 클라이언트 ID | `404k07uivdbplef5vp40bl0se` |
| `cognito_auth_scope` | OAuth 권한 범위 | `ecommerce-api/read` |
| `cognito_token_url` | 토큰 발급 엔드포인트 | `https://{domain}.auth.{region}.amazoncognito.com/oauth2/token` |

### OAuth 2.0 Scope 구조

권한 범위(scope)는 `{Resource Server ID}/{Scope Name}` 형식으로 구성됩니다:

```
default-m2m-resource-server-816839f0/read
└──────────────┬─────────────────────┘ └─┬─┘
         Resource Server ID            Scope
```

| 구성 요소 | 설명 |
|-----------|------|
| `default-m2m-resource-server` | Machine-to-Machine(M2M) 리소스 서버 기본 이름 |
| `816839f0` | CloudFormation이 추가한 고유 접미사 (중복 방지) |
| `/read` | 이 리소스 서버에서 정의된 권한 범위 |

### Cognito Resource Server란?

```
┌─────────────────────────────────────────────────────────────┐
│                    Cognito User Pool                         │
│                                                             │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  Resource Server: default-m2m-resource-server-xxx   │   │
│  │                                                     │   │
│  │  Scopes:                                            │   │
│  │    • /read  - API 읽기 권한                         │   │
│  │    • /write - API 쓰기 권한 (선택)                  │   │
│  └─────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
```

- **M2M (Machine-to-Machine)**: 사용자 개입 없이 서버 간 통신에 사용
- **Resource Server**: 보호할 API/리소스를 정의
- **Scope**: 클라이언트가 요청할 수 있는 권한 수준

> **💡 참고**: Gateway는 JWT 토큰의 `scope` 클레임을 검증하여 클라이언트가 해당 도구에 접근할 권한이 있는지 확인합니다.

### MCPClient 구성

```python
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],  # Gateway MCP 엔드포인트
        headers={"Authorization": f"Bearer {access_token}"},  # JWT 토큰
    )
)
```

> **💡 참고**: `streamablehttp_client`는 HTTP 기반 MCP 프로토콜을 지원하며, Gateway와의 양방향 통신을 처리합니다.

In [ ]:
# OAuth 인증 정보 조회
client_id = get_ssm_parameter("/app/ecommerce/agentcore/machine_client_id")
client_secret = get_cognito_client_secret()
scope = get_ssm_parameter("/app/ecommerce/agentcore/cognito_auth_scope")
token_url = get_ssm_parameter("/app/ecommerce/agentcore/cognito_token_url")

print("🔐 OAuth 인증 설정")
print("=" * 60)
print(f"   클라이언트 ID:     {client_id}")
print(f"   클라이언트 시크릿: {client_secret[:10]}...****")
print(f"   권한 범위:         {scope}")
print(f"   토큰 URL:          {token_url}")
print(f"   인증 서버:         Amazon Cognito ({REGION})")

print(f"\n🌐 Gateway 엔드포인트")
print("=" * 60)
print(f"   Gateway ID:  {gateway['id']}")
print(f"   Gateway URL: {gateway['gateway_url']}")

# 토큰 요청
print(f"\n🔑 액세스 토큰 요청 중...")
gateway_access_token = get_token(
    client_id,
    client_secret,
    scope,
    token_url
)

if 'access_token' in gateway_access_token:
    expires_in = gateway_access_token.get('expires_in', 'N/A')
    print(f"   ✅ 토큰 발급 성공!")
    print(f"   유효 기간:  {expires_in}초 ({int(expires_in/60)}분)")
    print(f"   토큰 타입:  Bearer")
    print(f"   토큰 미리보기: {gateway_access_token['access_token'][:50]}...")
else:
    print(f"   ❌ 토큰 발급 실패: {gateway_access_token.get('error', 'Unknown error')}")

# MCP 클라이언트 설정
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

print(f"\n✅ MCP 클라이언트 설정 완료!")
print(f"   이제 에이전트가 Gateway의 도구를 사용할 수 있습니다.")

## Step 7.2. MCP 클라이언트를 사용하여 에이전트에서 도구 액세스
이제 구축한 AgentCore Gateway와 이전 랩의 리소스를 함께 사용하여 Strands Agent를 생성하겠습니다. 우리 에이전트는 이제 Strands Agent를 통한 로컬 도구와 AgentCore Gateway를 통한 MCP 도구의 혼합을 사용합니다

In [ ]:
# Lab 1과 Lab 2에서 생성한 도구와 메모리 가져오기
import sys
sys.path.insert(0, '../src')

from agent import (
    SYSTEM_PROMPT,
    process_return,
    process_exchange, 
    web_search,
    MODEL_ID
)
print("✅ agent 모듈 import 완료")

from helpers.ecommerce_memory import (
    EcommerceCustomerMemoryHooks,
    create_or_get_ecommerce_memory_resource
)
import uuid
from bedrock_agentcore.memory import MemoryClient

memory_client = MemoryClient(region_name=REGION)

memory_id = create_or_get_ecommerce_memory_resource()
SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_ecommerce_001"

print(f"\n🔧 설정된 변수:")
print(f"   REGION: {REGION}")
print(f"   MODEL_ID: {MODEL_ID}")
print(f"   Memory ID: {memory_id}")
print(f"   Session ID: {SESSION_ID}")
print(f"   Customer ID: {CUSTOMER_ID}")

memory_hooks = EcommerceCustomerMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Bedrock 모델 초기화
model = BedrockModel(
    model_id=MODEL_ID,
    temperature=0.3,
    region_name=REGION
)
print(f"\n🤖 Bedrock 모델 초기화 완료")

# MCP 클라이언트 초기화
print(f"\n🔗 MCP 클라이언트 연결 중...")
print(f"   Gateway URL: {gateway['gateway_url']}")

try:
    # 이미 실행 중이면 재시작
    try:
        mcp_client.stop()
    except:
        pass
    mcp_client.start()
    mcp_tools = mcp_client.list_tools_sync()
    print(f"✅ MCP 클라이언트 연결 성공! 사용 가능한 도구: {len(mcp_tools)}개")
    for tool in mcp_tools:
        # MCPAgentTool 객체의 속성 확인
        tool_name = getattr(tool, 'tool_name', None) or getattr(tool, 'name', str(tool))
        tool_desc = getattr(tool, 'tool_description', None) or getattr(tool, 'description', '')
        if tool_desc:
            print(f"   • {tool_name}: {tool_desc[:50]}...")
        else:
            print(f"   • {tool_name}")
except Exception as e:
    print(f"⚠️ MCP 클라이언트 연결 실패: {str(e)}")
    print("💡 로컬 도구만 사용하여 에이전트를 생성합니다.")
    mcp_tools = []

# 로컬 도구와 MCP 도구 결합
tools = [
    process_return,      # 로컬 도구: 반품 처리
    process_exchange,    # 로컬 도구: 교환 처리
    web_search,          # 로컬 도구: 웹 검색
] + mcp_tools

print(f"\n📋 총 도구 개수: {len(tools)}개")
print("   로컬 도구:")
print("   • process_return - 반품 처리")
print("   • process_exchange - 교환 처리")
print("   • web_search - 웹 검색")
if mcp_tools:
    print(f"   MCP 도구: {len(mcp_tools)}개")
    for tool in mcp_tools:
        tool_name = getattr(tool, 'tool_name', None) or getattr(tool, 'name', str(tool))
        print(f"   • {tool_name}")

# 이커머스 고객 지원 에이전트 생성
agent = Agent(
    model=model,
    tools=tools,
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("\n✅ 이커머스 고객 지원 에이전트가 성공적으로 생성되었습니다!")

## Step 8: MCP 도구 액세스로 에이전트 테스트"

모든 기능이 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해보겠습니다.

In [ ]:
test_prompts = [
    # MCP 도구 테스트 (check_return_eligibility)
    "주문번호 KS-2024-001234 반품 가능한지 확인해주세요",
    
    # 로컬 도구 테스트 (process_exchange)
    "청바지를 M에서 L로 교환하고 싶어요. 주문번호는 KS-2024-005678이에요.",
]

# 에이전트 테스트 함수
def test_agent(agent, prompt, test_num):
    print(f"\n{'='*60}")
    print(f"테스트 {test_num}: {prompt}")
    print('='*60)
    
    try:
        response = agent(prompt)
        
        # 응답에서 처음 500자만 출력 (간결하게)
        response_text = str(response)
        if len(response_text) > 500:
            print(f"\n📝 응답 (처음 500자):\n{response_text[:500]}...")
        else:
            print(f"\n📝 응답:\n{response_text}")
            
    except Exception as e:
        print(f"❌ 오류: {str(e)}")

# 테스트 실행
print("🧪 이커머스 고객 지원 에이전트 테스트")
print("   • 로컬 도구: process_return, process_exchange, web_search")
print(f"   • MCP 도구: {len(mcp_tools)}개" if mcp_tools else "   • MCP 도구: 없음")

for i, prompt in enumerate(test_prompts, 1):
    test_agent(agent, prompt, i)

print(f"\n{'='*60}")
print("✅ 테스트 완료!")
print('='*60)

### 축하합니다! 🎉

**Lab 3: AgentCore Gateway로 도구를 에이전트에 안전하게 연결**을 성공적으로 완료했습니다

달성한 것:

##### 도구 중앙화 및 재사용성:

- 반품 자격 검증을 로컬 도구에서 중앙화된 AgentCore Gateway로 마이그레이션
- 기존 엔터프라이즈 Lambda 함수 통합 (반품 자격 검증)
- 여러 에이전트 유형이 액세스할 수 있는 공유 도구 인프라 생성

##### 엔터프라이즈급 보안:

- Cognito 통합으로 JWT 기반 인증 구현
- 게이트웨이 액세스를 위한 보안 인바운드 인증 구성
- 도구 사용을 위한 신원 기반 액세스 제어 설정

##### 확장 가능한 아키텍처 기반:

- 여러 사용 사례를 제공하는 재사용 가능한 도구 구축 (이커머스 고객 지원, 판매, 반품 처리)
- 다른 에이전트 간 코드 중복 제거
- 도구 업데이트 및 유지 관리를 위한 중앙화된 관리 생성


##### 현재 한계점 (다음에 해결 예정!):

- **로컬 개발 환경** - 아직 노트북에서 실행, 프로덕션 준비 안됨
- **제한된 관측성** - 에이전트 동작 및 성능에 대한 포괄적인 모니터링 없음
- **수동 확장** - 증가된 부하나 여러 동시 사용자를 자동으로 처리할 수 없음

##### 다음 단계: Lab 4 - AgentCore Runtime으로 프로덕션 배포

Lab 4에서는 프로토타입을 다음과 같은 프로덕션 준비 시스템으로 변환하겠습니다:

- 확장 가능한 에이전트 배포를 위한 AgentCore Runtime
- 메트릭, 로깅 및 추적을 통한 포괄적인 관측성
- 실제 트래픽을 처리하는 자동 확장 기능

### 리소스
- [Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)
- [Strands Agents 문서](https://github.com/strands-agents/sdk-python)
- [공식 이커머스 고객 지원 샘플](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)